# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

dataset_path = ""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
#     print(dirname)
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        dataset_path = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
model_directory = "/kaggle/input/segment-anything/pytorch/vit-h/1"
import sys
sys.path.append(model_directory)

In [3]:
dataset_directory = "/kaggle/input/fitzpatrick/CategoizedFitz"
import sys
sys.path.append(dataset_directory)

In [4]:
import glob
for files in glob.glob(model_directory + "/*.*"):
    print(files)

/kaggle/input/segment-anything/pytorch/vit-h/1/model.pth


In [5]:
print(sys.path)

['/kaggle/working', '/kaggle/lib/kagglegym', '/kaggle/lib', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages', '/root/src/BigQuery_Helper', '/kaggle/input/segment-anything/pytorch/vit-h/1', '/kaggle/input/fitzpatrick/CategoizedFitz']


In [6]:
import matplotlib.pyplot as plt

In [7]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [8]:
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint="/kaggle/input/segment-anything/pytorch/vit-h/1/model.pth")
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=100,
    pred_iou_thresh=0.90,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing,
    output_mode="binary_mask",
    
    
    
)

In [9]:
import cv2
image = cv2.imread("/kaggle/input/fitzpatrick/CategorizedFitz/eczema/05f2225244026969ba7435f5959acedf.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image

array([[[239, 130,  99],
        [255, 154, 124],
        [239, 140, 108],
        ...,
        [241, 104,  94],
        [243, 111,  99],
        [255, 136, 124]],

       [[238, 138, 104],
        [246, 147, 115],
        [240, 146, 112],
        ...,
        [255, 123, 111],
        [249, 115, 104],
        [255, 124, 113]],

       [[243, 149, 113],
        [240, 146, 112],
        [254, 164, 129],
        ...,
        [255, 132, 119],
        [248, 114, 102],
        [248, 112, 100]],

       ...,

       [[255, 127, 105],
        [255, 127, 105],
        [249, 120,  98],
        ...,
        [145,  77,  68],
        [146,  76,  68],
        [144,  74,  66]],

       [[245, 116,  94],
        [246, 117,  95],
        [240, 111,  89],
        ...,
        [145,  77,  68],
        [146,  76,  68],
        [142,  72,  64]],

       [[236, 110,  87],
        [252, 126, 103],
        [255, 131, 108],
        ...,
        [144,  76,  67],
        [143,  75,  66],
        [141,  73,  64]]

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

In [128]:
masks = mask_generator.generate(image)

In [ ]:
masks

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2

# Assuming 'data' contains your segmentation data
data = masks

# Thresholds for filtering masks
lower_threshold = 0.0001  # Discard masks with less than 5% True values
upper_threshold = 0.95  # Discard masks with more than 95% True values

# Filter out almost pure white or pure black masks
filtered_masks = [segment['segmentation'] for segment in data if lower_threshold < np.mean(segment['segmentation']) < upper_threshold]

# Combine all remaining masks using OR operator
if filtered_masks:
    combined_mask_or = np.logical_or.reduce(filtered_masks)
    # Convert the combined mask to 8-bit unsigned integer format for saving
    combined_mask_uint8 = (combined_mask_or * 255).astype(np.uint8)
    
    # Save the combined mask as a PNG file using OpenCV
    cv2.imwrite('/kaggle/working/combined_mask_or.png', combined_mask_uint8)
else:
    print("No masks remaining after filtering.")
    # Handle the case where no masks are left after filtering

# Assuming all segmentations are for the same image size, get image dimensions from one of the filtered masks
if filtered_masks:
    image_height, image_width = filtered_masks[0].shape

    # Create a figure for displaying the combined mask and the individual masks
    fig, axs = plt.subplots(len(filtered_masks) + 1, 1, figsize=(5, (len(filtered_masks) + 1) * 5))  # +1 for the combined mask

    # Check if axs is not an array (happens when len(filtered_masks) == 1)
    if not isinstance(axs, np.ndarray):
        axs = [axs]

    # Display combined mask in the first subplot
    axs[0].imshow(combined_mask_or, cmap='gray')
    axs[0].set_title('Combined Mask')
    axs[0].axis('off')

    # Loop through each filtered mask and display it
    for ax, mask in zip(axs[1:], filtered_masks):  # Skip the first ax intended for the combined mask
        ax.imshow(mask, cmap='gray')
        ax.axis('off')  # Remove axes for clarity

    plt.tight_layout()
    plt.show()


NameError: name 'masks' is not defined

In [14]:
!pip install Flask==3.0.0 pyngrok==7.1.2

  Obtaining dependency information for pyngrok==7.1.2 from https://files.pythonhosted.org/packages/2a/fa/131a4d290c7c98293ea812d4f50c40fc5c18640e92073698021f63576fa0/pyngrok-7.1.2-py3-none-any.whl.metadata


In [137]:
from flask import Flask, request, send_file, jsonify
import numpy as np
from io import BytesIO
from PIL import Image
import os

In [138]:
ngrok_key = "2YEEfAspjAhS7hCF9XvndSLiUQP_7fgVsnfKCFY7D66LJGKxy"
port = 5000

In [139]:
from pyngrok import ngrok

In [140]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://68f9-35-204-121-135.ngrok-free.app'

In [141]:
app = Flask(__name__)

In [142]:
@app.route("/")
def hello():
  return "Hello World!! from anywhere in the world!"


In [143]:

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        # Read the file into a numpy array using OpenCV
        filestr = file.read()
        # Convert the string data to numpy array
        npimg = np.frombuffer(filestr, np.uint8)
        # Load the numpy array as an OpenCV image
        img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

        # Process the image (here, we just save it as is)
        save_path = '/kaggle/working/temp_image.png'  # Specify the path and filename
        cv2.imwrite(save_path, img)
        
        
        image = cv2.imread(save_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        masks = mask_generator.generate(image)
        data = masks

        # Thresholds for filtering masks
        lower_threshold = 0.0001  # Discard masks with less than 5% True values
        upper_threshold = 0.95  # Discard masks with more than 95% True values
        
            # Check if thresholds are provided in the form data and update accordingly
        if 'lower_threshold' in request.form:
            try:
                lower_threshold = float(request.form['lower_threshold'])
            except ValueError:
                return 'Invalid lower_threshold value', 400
        if 'upper_threshold' in request.form:
            try:
                upper_threshold = float(request.form['upper_threshold'])
            except ValueError:
                return 'Invalid upper_threshold value', 400
        # Filter out almost pure white or pure black masks
        filtered_masks = [segment['segmentation'] for segment in data if lower_threshold < np.mean(segment['segmentation']) < upper_threshold]

        # Combine all remaining masks using OR operator
        if filtered_masks:
            combined_mask_or = np.logical_or.reduce(filtered_masks)
            # Convert the combined mask to 8-bit unsigned integer format for saving
            combined_mask_uint8 = (combined_mask_or * 255).astype(np.uint8)
            cv2.imwrite('/kaggle/working/combined_mask_or.png', combined_mask_uint8)

        # Here, you can process the image if necessary

        # For simplicity, we return the same image
        return send_file("/kaggle/working/combined_mask_or.png", mimetype='image/png')

In [ ]:
if __name__ == '__main__':
  app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


In [111]:
print(data)

NameError: name 'data' is not defined

In [ ]:
data = masks

# Thresholds for filtering masks
lower_threshold = 0.0001  # Discard masks with less than 5% True values
upper_threshold = 0.95  # Discard masks with more than 95% True values

# Filter out almost pure white or pure black masks
filtered_masks = [segment['segmentation'] for segment in data if lower_threshold < np.mean(segment['segmentation']) < upper_threshold]

# Combine all remaining masks using OR operator
if filtered_masks:
    combined_mask_or = np.logical_or.reduce(filtered_masks)
    # Convert the combined mask to 8-bit unsigned integer format for saving
    combined_mask_uint8 = (combined_mask_or * 255).astype(np.uint8)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
len(masks)


In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_output(masks)
plt.axis('off')
plt.savefig("/kaggle/working/output.png")
plt.show() 

In [ ]:
dataset_path = ""
import os

for dirname, _, filenames in os.walk('/kaggle/input/fitzpatrick/CategorizedFitz'):
    
    
    if dirname != "/kaggle/input/fitzpatrick/CategorizedFitz":
        print(dirname)
        
        temp_dirname = "/kaggle/working" + dirname + "_SAM"
        if not os.path.exists(temp_dirname):
            os.makedirs(temp_dirname, exist_ok=False)
            print("made new dir" + temp_dirname)
        for filename in filenames:
            temp_filename = temp_dirname + "/" + filename
#         print(os.path.join(dirname, filename))
            file_path = os.path.join(dirname, filename)
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            masks = mask_generator.generate(image)
            plt.figure(figsize=(20,20))
            plt.imshow(image)
            show_anns(masks)
            print(temp_filename)
            plt.axis('off')
            plt.savefig(temp_filename)
            plt.show()
            plt.close()
            
            


In [ ]:
images = glob.glob('/kaggle/input/fitzpatrick/*/')[0]
label_151 = ['pediculosis lids', 'dermatomyositis', 'urticaria', 'eczema']
#Entirely used to make_directory of working classes
working_path = "/kaggle/working/working_classes"
import shutil
if not(os.path.exists(working_path)):
    os.mkdir(working_path)
for label in label_151:
    new_dir = working_path + "/" + label
    shutil.copytree(images + "/" + label, new_dir)
    
workings = glob.glob(working_path + "/*/")


In [ ]:
workings

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# Ensure output directory exists
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Your workings list
workings = [
    '/kaggle/working/working_classes/eczema/',
    '/kaggle/working/working_classes/acne/',
    '/kaggle/working/working_classes/pediculosis lids/',
    
]

# Thresholds for filtering masks
lower_threshold = 0.001  # Lower threshold for filtering
upper_threshold = 0.85   # Upper threshold for filtering

for dir_path in workings:
    disease_name = os.path.basename(os.path.normpath(dir_path))
    output_dir_path = os.path.join('/kaggle/working/output', disease_name)
    
    if os.path.exists(dir_path):
        for filename in os.listdir(dir_path):
            file_path = os.path.join(dir_path, filename)
            
            if os.path.isfile(file_path):
                image = cv2.imread(file_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                if image is not None:
                    masks = mask_generator.generate(image)  # Assuming mask_generator.generate() returns a list of dicts
                    
                    filtered_masks = [mask['segmentation'] for mask in masks if lower_threshold < np.mean(mask['segmentation']) < upper_threshold]
                    
                    if filtered_masks:
                        combined_mask_or = np.logical_or.reduce(filtered_masks)
                        combined_mask_uint8 = (combined_mask_or * 255).astype(np.uint8)
                        
                        # Ensure the output directory exists
                        ensure_dir(output_dir_path)
                        
                        # Visualize the original image and the combined mask side by side
                        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                        ax[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                        ax[0].set_title(f"Original Image: {disease_name}")
                        ax[0].axis('off')
                        
                        ax[1].imshow(combined_mask_uint8, cmap='gray')
                        ax[1].set_title("Combined Mask")
                        ax[1].axis('off')
                        
                        plt.show()
                        
                        # Save the combined mask in the specified output directory
                        mask_file_path = os.path.join(output_dir_path, f"mask_{filename}")
                        cv2.imwrite(mask_file_path, combined_mask_uint8)
                        print(f"Saved mask: {mask_file_path}")
                    else:
                        print(f"No valid masks found for {file_path}. Skipping.")
                else:
                    print(f"Failed to load image: {file_path}")
    else:
        print(f"Directory does not exist: {dir_path}")
